In [1]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

В предыдущем пункте мы ознакомились с данными, открыв 1/12 часть данных, произвели ее "горячее кодирование" и группировку с агрегацией разными функциями для определенных колонок, чем сгенерировали, можно сказать, дополнительные признаки. Также было сгенерировано два отдельных признака, благодаря которым метрика чуть-чуть повысилась.

Теперь пришло время произвести все те же действия, но на полном датафрейме. Сложность в том, что данных очень много, а ресурсы оперативной памяти ограничены. Как данная проблема была решена?
- В первую очередь, при открытии parquet-файлов колонки были загружены в таком типе данных, который позволял минимально расходовать память и при этом не искажал данные. Лишь колонка 'id' потребовала тип 'int32', все остальные имели малые целочисленные значения, и было достаточно типа данных 'int8'.
- Все 12 датафрейсмов были соединены по горизонтали. Датафрейм размерностью 26млн строк на 61 столбец занял объем 1,6Гб.
- OneHotEncoder используем с гиперпараметрами min_frequеncy=3000 (снижаем вероятность переобучения и размерность будущего датафрейма и памяти) и dtype='int8' (снижаем размер потребляемой памяти). Категории, которые встречаются с частотой менее 3000 раз, будут помещены в отдельную категорию.  
- Больше всего ресурсов потребовала агрегация признаков по 'id'. Проблема была решена путем создания "файла подкачки" нужного (большого) размера. 
- Последний пункт: соединение с целевой переменной. В итоге имеем финальный датафрейм размерностью 3млн строк и 361 столбец и объемом 1,3Гб. Сохраняем в parguet-файл (таким образом типы колонок остануться прежними). Готовы к моделированию.

In [2]:
dict_types = {'id':'int32', 'rn':'int8', 'pre_since_opened':'int8', 'pre_since_confirmed':'int8', 'pre_pterm':'int8',
              'pre_fterm':'int8', 'pre_till_pclose':'int8', 'pre_till_fclose':'int8', 'pre_loans_credit_limit':'int8',
              'pre_loans_next_pay_summ':'int8', 'pre_loans_outstanding':'int8', 'pre_loans_total_overdue':'int8', 
              'pre_loans_max_overdue_sum':'int8', 'pre_loans_credit_cost_rate':'int8', 'pre_loans5':'int8',
              'pre_loans530':'int8', 'pre_loans3060':'int8', 'pre_loans6090':'int8', 'pre_loans90':'int8',
              'is_zero_loans5':'int8', 'is_zero_loans530':'int8', 'is_zero_loans3060':'int8', 'is_zero_loans6090':'int8',
              'is_zero_loans90':'int8', 'pre_util':'int8', 'pre_over2limit':'int8', 'pre_maxover2limit':'int8',
              'is_zero_util':'int8', 'is_zero_over2limit':'int8', 'is_zero_maxover2limit':'int8', 'enc_paym_0':'int8',
              'enc_paym_1':'int8', 'enc_paym_2':'int8', 'enc_paym_3':'int8', 'enc_paym_4':'int8', 'enc_paym_5':'int8',
              'enc_paym_6':'int8', 'enc_paym_7':'int8', 'enc_paym_8':'int8', 'enc_paym_9':'int8', 'enc_paym_10':'int8',
              'enc_paym_11':'int8', 'enc_paym_12':'int8', 'enc_paym_13':'int8', 'enc_paym_14':'int8', 'enc_paym_15':'int8',
              'enc_paym_16':'int8', 'enc_paym_17':'int8', 'enc_paym_18':'int8', 'enc_paym_19':'int8', 'enc_paym_20':'int8',
              'enc_paym_21':'int8', 'enc_paym_22':'int8', 'enc_paym_23':'int8', 'enc_paym_24':'int8',
              'enc_loans_account_holder_type':'int8', 'enc_loans_credit_status':'int8', 'enc_loans_credit_type':'int8',
              'enc_loans_account_cur':'int8', 'pclose_flag':'int8', 'fclose_flag':'int8'}

In [3]:
%%time
df_0 = pd.read_parquet('train_data/train_data_0.pq').astype(dict_types)
df_1 = pd.read_parquet('train_data/train_data_1.pq').astype(dict_types)
df_2 = pd.read_parquet('train_data/train_data_2.pq').astype(dict_types)
df_3 = pd.read_parquet('train_data/train_data_3.pq').astype(dict_types)
df_4 = pd.read_parquet('train_data/train_data_4.pq').astype(dict_types)
df_5 = pd.read_parquet('train_data/train_data_5.pq').astype(dict_types)
df_6 = pd.read_parquet('train_data/train_data_6.pq').astype(dict_types)
df_7 = pd.read_parquet('train_data/train_data_7.pq').astype(dict_types)
df_8 = pd.read_parquet('train_data/train_data_8.pq').astype(dict_types)
df_9 = pd.read_parquet('train_data/train_data_9.pq').astype(dict_types)
df_10 = pd.read_parquet('train_data/train_data_10.pq').astype(dict_types)
df_11 = pd.read_parquet('train_data/train_data_11.pq').astype(dict_types)

CPU times: total: 1min 15s
Wall time: 10.6 s


In [4]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974724 entries, 0 to 1974723
Data columns (total 61 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int32
 1   rn                             int8 
 2   pre_since_opened               int8 
 3   pre_since_confirmed            int8 
 4   pre_pterm                      int8 
 5   pre_fterm                      int8 
 6   pre_till_pclose                int8 
 7   pre_till_fclose                int8 
 8   pre_loans_credit_limit         int8 
 9   pre_loans_next_pay_summ        int8 
 10  pre_loans_outstanding          int8 
 11  pre_loans_total_overdue        int8 
 12  pre_loans_max_overdue_sum      int8 
 13  pre_loans_credit_cost_rate     int8 
 14  pre_loans5                     int8 
 15  pre_loans530                   int8 
 16  pre_loans3060                  int8 
 17  pre_loans6090                  int8 
 18  pre_loans90                    int8 
 19  

In [5]:
df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11], ignore_index=True)

In [6]:
df

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26162712,2999999,8,6,5,14,13,1,15,16,2,...,0,0,0,1,1,3,4,1,0,0
26162713,2999999,9,5,3,2,10,15,14,17,2,...,0,0,0,4,1,3,4,1,0,0
26162714,2999999,10,3,16,11,13,14,8,15,5,...,0,0,3,4,1,2,4,1,0,0
26162715,2999999,11,3,6,4,8,1,11,0,5,...,3,3,3,4,1,2,3,1,1,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26162717 entries, 0 to 26162716
Data columns (total 61 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int32
 1   rn                             int8 
 2   pre_since_opened               int8 
 3   pre_since_confirmed            int8 
 4   pre_pterm                      int8 
 5   pre_fterm                      int8 
 6   pre_till_pclose                int8 
 7   pre_till_fclose                int8 
 8   pre_loans_credit_limit         int8 
 9   pre_loans_next_pay_summ        int8 
 10  pre_loans_outstanding          int8 
 11  pre_loans_total_overdue        int8 
 12  pre_loans_max_overdue_sum      int8 
 13  pre_loans_credit_cost_rate     int8 
 14  pre_loans5                     int8 
 15  pre_loans530                   int8 
 16  pre_loans3060                  int8 
 17  pre_loans6090                  int8 
 18  pre_loans90                    int8 
 19

In [8]:
# Проверка на строки-дубликаты.
len(df[df.duplicated()])

0

In [9]:
# Проверка на пропуски.
df.isna().sum().sort_values(ascending=False)

id                    0
enc_paym_1            0
enc_paym_3            0
enc_paym_4            0
enc_paym_5            0
                     ..
pre_over2limit        0
pre_maxover2limit     0
is_zero_util          0
is_zero_over2limit    0
fclose_flag           0
Length: 61, dtype: int64

In [9]:
binary_cols = ['pre_since_opened', 'pre_since_confirmed', 'pre_pterm', 'pre_fterm', 'pre_till_pclose', 'pre_till_fclose', 
               'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding', 'pre_loans_total_overdue', 
               'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5', 'pre_loans530', 'pre_loans3060',
               'pre_loans6090', 'pre_loans90', 'pre_util', 'pre_over2limit','pre_maxover2limit']

for column in binary_cols:    
    column_name = df[column]
    print(f"Характеристика: {column} \nКоличество уникальных значений: {column_name.nunique()} \nСписок значений: \n{column_name.value_counts(dropna=False)} \n")

Характеристика: pre_since_opened 
Количество уникальных значений: 20 
Список значений: 
13    1645938
0     1561873
6     1522882
10    1409770
8     1406244
18    1388400
14    1315095
9     1312790
7     1307521
1     1300842
2     1290724
4     1255220
11    1250266
19    1230414
3     1211933
17    1202495
16    1200566
5     1196677
15    1193193
12     959874
Name: pre_since_opened, dtype: int64 

Характеристика: pre_since_confirmed 
Количество уникальных значений: 18 
Список значений: 
9     4909419
6     1948337
4     1514861
17    1503451
3     1435553
14    1410504
10    1361991
13    1313856
7     1291490
1     1245131
2     1239514
8     1234641
16    1213550
0     1192383
5     1180787
12    1108831
11    1058230
15        188
Name: pre_since_confirmed, dtype: int64 

Характеристика: pre_pterm 
Количество уникальных значений: 18 
Список значений: 
4     4456185
14    2336504
2     1532229
17    1520418
1     1483207
16    1460983
11    1430459
9     1428200
6     1400005
7

In [10]:
# Удаляем колонку, состоящую из одного значения.
df.drop(['pre_loans_total_overdue'], axis=1, inplace=True)

# Перезаписываем список бинаризированных колонок.
binary_cols = ['pre_since_opened', 'pre_since_confirmed', 'pre_pterm', 'pre_fterm', 'pre_till_pclose', 'pre_till_fclose', 
               'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding', 'pre_loans_max_overdue_sum',
               'pre_loans_credit_cost_rate', 'pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
               'pre_util', 'pre_over2limit','pre_maxover2limit']

In [11]:
coded_cols = ['enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7',
              'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14',
              'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20', 'enc_paym_21',
              'enc_paym_22', 'enc_paym_23', 'enc_paym_24', 'enc_loans_account_holder_type', 'enc_loans_credit_status',
              'enc_loans_credit_type', 'enc_loans_account_cur']

for column in coded_cols:    
    column_name = df[column]
    print(f"Характеристика: {column} \nКоличество уникальных значений: {column_name.nunique()} \nСписок значений: \n{column_name.value_counts(dropna=False)} \n")

Характеристика: enc_paym_0 
Количество уникальных значений: 4 
Список значений: 
0    24136144
3     1109198
1      853844
2       63531
Name: enc_paym_0, dtype: int64 

Характеристика: enc_paym_1 
Количество уникальных значений: 4 
Список значений: 
0    21792098
3     2733662
1     1521366
2      115591
Name: enc_paym_1, dtype: int64 

Характеристика: enc_paym_2 
Количество уникальных значений: 4 
Список значений: 
0    20686082
3     4008468
1     1362284
2      105883
Name: enc_paym_2, dtype: int64 

Характеристика: enc_paym_3 
Количество уникальных значений: 4 
Список значений: 
0    19601983
3     5167483
1     1292438
2      100813
Name: enc_paym_3, dtype: int64 

Характеристика: enc_paym_4 
Количество уникальных значений: 4 
Список значений: 
0    18500050
3     6356585
1     1212916
2       93166
Name: enc_paym_4, dtype: int64 

Характеристика: enc_paym_5 
Количество уникальных значений: 4 
Список значений: 
0    17409261
3     7542048
1     1125455
2       85953
Name: enc_pay

In [12]:
flag_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 'is_zero_util',
             'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag' ]

for column in flag_cols:    
    column_name = df[column]
    print(f"Характеристика: {column} \nКоличество уникальных значений: {column_name.nunique()} \nСписок значений: \n{column_name.value_counts(dropna=False)} \n")

Характеристика: is_zero_loans5 
Количество уникальных значений: 2 
Список значений: 
1    24035861
0     2126856
Name: is_zero_loans5, dtype: int64 

Характеристика: is_zero_loans530 
Количество уникальных значений: 2 
Список значений: 
1    21772900
0     4389817
Name: is_zero_loans530, dtype: int64 

Характеристика: is_zero_loans3060 
Количество уникальных значений: 2 
Список значений: 
1    25047143
0     1115574
Name: is_zero_loans3060, dtype: int64 

Характеристика: is_zero_loans6090 
Количество уникальных значений: 2 
Список значений: 
1    25519933
0      642784
Name: is_zero_loans6090, dtype: int64 

Характеристика: is_zero_loans90 
Количество уникальных значений: 2 
Список значений: 
1    25535248
0      627469
Name: is_zero_loans90, dtype: int64 

Характеристика: is_zero_util 
Количество уникальных значений: 2 
Список значений: 
1    18258067
0     7904650
Name: is_zero_util, dtype: int64 

Характеристика: is_zero_over2limit 
Количество уникальных значений: 2 
Список значений

In [13]:
ohe = OneHotEncoder(sparse_output=False, min_frequency=3000, dtype='int8')

In [14]:
%%time
ohe_cols = binary_cols + coded_cols

ohe_data = pd.DataFrame(ohe.fit_transform(df[ohe_cols]), columns=ohe.get_feature_names_out())
ohe_data.shape

CPU times: total: 1min 34s
Wall time: 2min 27s


(26162717, 348)

In [15]:
df = df.join(ohe_data).drop(columns=ohe_cols)
print(df.shape)
df.head()

(26162717, 360)


,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,...,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_infrequent_sklearn
0,0,1,1,1,1,1,1,1,1,1,...,0,0,1,0,0,0,0,1,0,0
1,0,2,1,1,1,1,1,1,1,1,...,0,0,1,0,0,0,0,1,0,0
2,0,3,1,1,1,1,1,0,1,1,...,0,1,0,0,0,0,0,1,0,0
3,0,4,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0
4,0,5,1,1,1,1,1,1,1,1,...,0,0,1,0,0,0,0,1,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26162717 entries, 0 to 26162716
Columns: 360 entries, id to enc_loans_account_cur_infrequent_sklearn
dtypes: int32(1), int8(359)
memory usage: 8.8 GB


In [17]:
%%time
category_df = df.groupby(['id'], as_index=False)[ohe.get_feature_names_out()].sum()
category_df

CPU times: total: 3min 21s
Wall time: 18min 14s


,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_infrequent_sklearn
0,0,0,1,1,1,1,2,0,1,0,...,0,2,7,0,0,0,0,10,0,0
1,1,0,0,1,0,0,0,0,1,2,...,0,3,8,0,0,0,0,14,0,0
2,2,1,0,0,0,0,0,0,0,0,...,0,2,1,0,0,0,0,3,0,0
3,3,0,3,1,0,2,1,3,0,0,...,0,4,9,1,0,0,0,15,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,1,2,0,0,2,0,1,0,0,...,0,4,6,0,0,1,0,11,0,0
2999996,2999996,0,1,0,1,1,5,0,0,1,...,0,4,8,0,0,1,0,13,0,0
2999997,2999997,0,0,1,1,3,0,0,0,0,...,0,3,7,0,0,0,0,10,0,0
2999998,2999998,0,0,0,0,1,0,0,1,0,...,0,2,2,0,0,0,0,5,0,0


In [18]:
rn_df = df.groupby(['id'], as_index=False)[['rn']].count()
rn_df

,id,rn
0,0,10
1,1,14
2,2,3
3,3,15
4,4,1
...,...,...
2999995,2999995,11
2999996,2999996,13
2999997,2999997,10
2999998,2999998,5


In [19]:
flag_df = df.groupby(['id'], as_index=False)[flag_cols].mean()
flag_df

,id,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag
0,0,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,0.100000,0.200000
1,1,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,0.071429,0.142857
2,2,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,0.666667,0.666667
3,3,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,0.333333,0.400000
4,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,0.818182,0.454545,1.000000,1.000000,1.000000,0.727273,1.000000,0.636364,0.181818,0.454545
2999996,2999996,0.923077,1.000000,0.923077,1.000000,1.000000,0.692308,0.846154,0.846154,0.384615,0.307692
2999997,2999997,0.900000,0.700000,1.000000,1.000000,1.000000,0.600000,1.000000,0.900000,0.100000,0.100000
2999998,2999998,1.000000,1.000000,0.800000,0.800000,0.800000,0.200000,0.800000,0.800000,0.400000,0.200000


In [20]:
df = pd.merge(left=rn_df, right=flag_df, on='id')
df

,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag
0,0,10,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,0.100000,0.200000
1,1,14,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,0.071429,0.142857
2,2,3,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,0.666667,0.666667
3,3,15,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,0.333333,0.400000
4,4,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,11,0.818182,0.454545,1.000000,1.000000,1.000000,0.727273,1.000000,0.636364,0.181818,0.454545
2999996,2999996,13,0.923077,1.000000,0.923077,1.000000,1.000000,0.692308,0.846154,0.846154,0.384615,0.307692
2999997,2999997,10,0.900000,0.700000,1.000000,1.000000,1.000000,0.600000,1.000000,0.900000,0.100000,0.100000
2999998,2999998,5,1.000000,1.000000,0.800000,0.800000,0.800000,0.200000,0.800000,0.800000,0.400000,0.200000


In [21]:
df = pd.merge(left=df, right=category_df, on='id')
df

,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,...,enc_loans_credit_type_2,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_infrequent_sklearn
0,0,10,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,...,0,2,7,0,0,0,0,10,0,0
1,1,14,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,...,0,3,8,0,0,0,0,14,0,0
2,2,3,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,...,0,2,1,0,0,0,0,3,0,0
3,3,15,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,...,0,4,9,1,0,0,0,15,0,0
4,4,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,11,0.818182,0.454545,1.000000,1.000000,1.000000,0.727273,1.000000,0.636364,...,0,4,6,0,0,1,0,11,0,0
2999996,2999996,13,0.923077,1.000000,0.923077,1.000000,1.000000,0.692308,0.846154,0.846154,...,0,4,8,0,0,1,0,13,0,0
2999997,2999997,10,0.900000,0.700000,1.000000,1.000000,1.000000,0.600000,1.000000,0.900000,...,0,3,7,0,0,0,0,10,0,0
2999998,2999998,5,1.000000,1.000000,0.800000,0.800000,0.800000,0.200000,0.800000,0.800000,...,0,2,2,0,0,0,0,5,0,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000000 entries, 0 to 2999999
Columns: 360 entries, id to enc_loans_account_cur_infrequent_sklearn
dtypes: float64(10), int32(1), int64(1), int8(348)
memory usage: 1.3 GB


In [23]:
target_df = pd.read_csv('train_data/train_target.csv').astype({'id':'int32', 'flag':'int8'})
print(target_df.shape)
target_df.head()

(3000000, 2)


,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [24]:
df = pd.merge(left=df, right=target_df, on='id')
df

,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,...,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,enc_loans_account_cur_0,enc_loans_account_cur_1,enc_loans_account_cur_2,enc_loans_account_cur_infrequent_sklearn,flag
0,0,10,0.900000,1.000000,1.000000,1.000000,1.000000,0.600000,0.900000,0.900000,...,2,7,0,0,0,0,10,0,0,0
1,1,14,0.857143,0.714286,0.857143,0.857143,0.785714,0.714286,0.857143,0.785714,...,3,8,0,0,0,0,14,0,0,0
2,2,3,1.000000,0.666667,0.666667,0.666667,1.000000,0.333333,1.000000,0.666667,...,2,1,0,0,0,0,3,0,0,0
3,3,15,1.000000,1.000000,1.000000,1.000000,1.000000,0.533333,0.933333,0.933333,...,4,9,1,0,0,0,15,0,0,0
4,4,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,2999995,11,0.818182,0.454545,1.000000,1.000000,1.000000,0.727273,1.000000,0.636364,...,4,6,0,0,1,0,11,0,0,0
2999996,2999996,13,0.923077,1.000000,0.923077,1.000000,1.000000,0.692308,0.846154,0.846154,...,4,8,0,0,1,0,13,0,0,0
2999997,2999997,10,0.900000,0.700000,1.000000,1.000000,1.000000,0.600000,1.000000,0.900000,...,3,7,0,0,0,0,10,0,0,0
2999998,2999998,5,1.000000,1.000000,0.800000,0.800000,0.800000,0.200000,0.800000,0.800000,...,2,2,0,0,0,0,5,0,0,0


In [25]:
# Checkpoint
df.to_parquet('train_data/df_final.parquet')